## Load the dataset


In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
np.random.seed(8)
import os
from google.colab import drive
drive.mount('/content/drive')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 5.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Mounted at /content/drive


In [ ]:
telemetry = pd.read_csv('/content/drive/MyDrive/aitomatic/PdM_telemetry.csv')
errors = pd.read_csv('/content/drive/MyDrive/aitomatic/PdM_errors.csv')
maint = pd.read_csv('/content/drive/MyDrive/aitomatic/PdM_maint.csv')
failures = pd.read_csv('/content/drive/MyDrive/aitomatic/PdM_failures.csv')
machines = pd.read_csv('/content/drive/MyDrive/aitomatic/PdM_machines.csv')
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'], format="%Y-%m-%d %H:%M:%S")
errors['datetime'] = pd.to_datetime(errors['datetime'], format="%Y-%m-%d %H:%M:%S")
maint['datetime'] = pd.to_datetime(maint['datetime'], format="%Y-%m-%d %H:%M:%S")
failures['datetime'] = pd.to_datetime(failures['datetime'], format="%Y-%m-%d %H:%M:%S")

In [ ]:
error_count = pd.get_dummies(errors)
features = telemetry.merge(error_count, on=['datetime', 'machineID'], how='left')
# comp_rep = pd.get_dummies(maint)
# features = features.merge(comp_rep, on=['datetime', 'machineID'], how='left')
machines_m = pd.get_dummies(machines)
features = features.merge(machines_m, on=['machineID'], how='left').fillna(0.0)
features = features.groupby(['machineID','datetime']).max().reset_index()
# features["hours"] = features['datetime'].apply(lambda x: x.hour / 24)
features = features.merge(failures, on=['datetime', 'machineID'], how='left').fillna('none')
label2int = {"none":0,"comp1":1,"comp2":1,"comp3":1,"comp4":1}
n_classes = len(label2int)
features["failure"] = features["failure"].apply(lambda x: label2int[x])
features

,machineID,datetime,volt,rotate,pressure,vibration,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5,age,model_model1,model_model2,model_model3,model_model4,failure
0,1,2015-01-01 06:00:00,176.217853,418.504078,113.077935,45.087686,0.0,0.0,0.0,0.0,0.0,18,0,0,1,0,0
1,1,2015-01-01 07:00:00,162.879223,402.747490,95.460525,43.413973,0.0,0.0,0.0,0.0,0.0,18,0,0,1,0,0
2,1,2015-01-01 08:00:00,170.989902,527.349825,75.237905,34.178847,0.0,0.0,0.0,0.0,0.0,18,0,0,1,0,0
3,1,2015-01-01 09:00:00,162.462833,346.149335,109.248561,41.122144,0.0,0.0,0.0,0.0,0.0,18,0,0,1,0,0
4,1,2015-01-01 10:00:00,157.610021,435.376873,111.886648,25.990511,0.0,0.0,0.0,0.0,0.0,18,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876137,100,2016-01-01 02:00:00,179.438162,395.222827,102.290715,50.771941,0.0,0.0,0.0,0.0,0.0,5,0,0,0,1,0
876138,100,2016-01-01 03:00:00,189.617555,446.207972,98.180607,35.123072,0.0,0.0,0.0,0.0,0.0,5,0,0,0,1,0
876139,100,2016-01-01 04:00:00,192.483414,447.816524,94.132837,48.314561,0.0,0.0,0.0,0.0,0.0,5,0,0,0,1,0
876140,100,2016-01-01 05:00:00,165.475310,413.771670,104.081073,44.835259,0.0,0.0,0.0,0.0,0.0,5,0,0,0,1,0


In [ ]:
#split train test
input_length = 24
n_upsampling = 5
hours = np.arange(input_length)
Q1, Q2, Q3 = input_length// 4, input_length // 2, input_length*3//4
milestone = '2015-10-20 01:00:00'
train = features[features["datetime"] < pd.to_datetime(milestone)]
test = features[features["datetime"] >= pd.to_datetime(milestone)]
# train.drop(columns=["datetime"],inplace=True)
# test.drop(columns=["datetime"],inplace=True)
x_train,y_train = [], []
x_test,y_test = [], []
#generate train
for group_name, df_group in train.groupby("machineID"):
  # print("user name",group_name)
  leng = len(df_group)
  start_ids = np.arange(0,leng-input_length+1,input_length)
  for id in start_ids:
    X = df_group.iloc[id:id+input_length,2:-1].values.astype(np.float32)
    y = df_group.iloc[id:id+input_length,-1]
    label = y.max()
    x_train.append(X)
    y_train.append(label)
    #generate more data
    if label > 0 and leng- id > input_length and id:
      pos_ids = hours[(y > 0).values]
      for sample_id in np.random.randint(pos_ids.min()-Q3,pos_ids.max()-Q1+1,n_upsampling):
        X = df_group.iloc[id+sample_id:id+sample_id+input_length,2:-1].values.astype(np.float32)
        y = df_group.iloc[id+sample_id:id+sample_id+input_length,-1]
        x_train.append(X)
        y_train.append(y.max())
  for id in np.random.randint(Q1,Q3+1,len(start_ids)-1):
    X = df_group.iloc[id:id+input_length,2:-1].values.astype(np.float32)
    y = df_group.iloc[id:id+input_length,-1].max()
    if y > 0:
      continue
    x_train.append(X)
    y_train.append(y)

#generate test
for group_name, df_group in test.groupby("machineID"):
  leng = len(df_group)
  start_ids = np.arange(0,leng-input_length+1,input_length)
  # slide_ids = np.arange(Q2,leng-input_length+1,input_length)
  # start_ids = list(start_ids) + list(slide_ids)
  for id in start_ids:
    X = df_group.iloc[id:id+input_length,2:-1].values.astype(np.float32)
    y = df_group.iloc[id:id+input_length,-1].max()
    x_test.append(X)
    y_test.append(y)

In [ ]:
# convert to 1 hot
x_test = np.array(x_test)
y_test_ = np.array(y_test)
y_test = np.zeros((y_test_.size, 2))
y_test[np.arange(y_test_.size), y_test_] = 1

x_train = np.array(x_train)
y_train_ = np.array(y_train)
y_train = np.zeros((y_train_.size, 2))
y_train[np.arange(y_train_.size), y_train_] = 1

In [ ]:
#show label distribution
len(y_train),len(y_test), np.unique(y_train.argmax(1),return_counts=True), np.unique(y_test.argmax(1),return_counts=True)

(55420,
 7300,
 (array([0, 1]), array([52007,  3413])),
 (array([0, 1]), array([7167,  133])))

## Build the model

Our model processes a tensor of shape `(batch size, sequence length, features)`,
where `sequence length` is the number of time steps and `features` is each input
timeseries.

You can replace your classification RNN layers with this one: the
inputs are fully compatible!

We include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.

The projection layers are implemented through `keras.layers.Conv1D`.

In [ ]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


In [ ]:

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


## Train and evaluate

In [ ]:

input_shape = x_train.shape[1:]
print(input_shape)
model = build_model(
    input_shape,
    head_size=128,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[64],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=[tfa.metrics.F1Score(num_classes=n_classes),'accuracy'],
)
model.summary()
checkpoint_path = "/content/drive/MyDrive/aitomatic/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
callbacks = [cp_callback]

model.fit(
    x_train,
    y_train,
    validation_data= (x_test,y_test),
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

(24, 20)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24, 20)]     0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 24, 20)      40          ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 24, 20)      42516       ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                   

KeyboardInterrupt: ignored

In [ ]:
metric = tfa.metrics.F1Score(num_classes=n_classes)
metric.update_state(y_test, model.predict(x_test))
result = metric.result()
result.numpy()


454/454 [==============================] - 3s 5ms/step


array([0.9907401 , 0.        , 0.05769231, 0.        , 0.02985075],
      dtype=float32)

In [ ]:
inputs = layers.Input(shape=x_train.shape[1:], name='input')

# x1 = layers.LSTM(units=50,return_sequences=True)(inputs)
# x1 = layers.Dropout(0.2)(x1)
# x1 = layers.LSTM(units=50,return_sequences=False)(inputs)
# x1 = layers.Dropout(0.2)(x1)

# outputs = layers.Dense(n_classes, activation="softmax")(x1)
x1 = layers.Bidirectional(layers.LSTM(units=50,return_sequences=False))(inputs)
x1 = layers.Dropout(0.2)(x1)
x2 = layers.Bidirectional(layers.LSTM(units=50,return_sequences=False))(inputs)
x2 = layers.Dropout(0.2)(x2)
# x3 = layers.Bidirectional(layers.LSTM(units=50,return_sequences=False))(inputs)
# x3 = layers.Dropout(0.2)(x3)
outputs = layers.Dense(n_classes, activation="softmax")(x2)

In [ ]:
model = keras.models.Model(inputs=inputs, outputs=outputs)

# model.save_weights('my_checkpoint')
checkpoint_path = "/content/drive/MyDrive/aitomatic/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,ave_weights_only=True,save_best_only=True,verbose=1)
callbacks = [cp_callback]
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=[tfa.metrics.F1Score(num_classes=n_classes)],
)
model.summary()
model.fit(
    x_train,
    y_train,
    validation_data= (x_test,y_test),
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 24, 14)]          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 100)              26000     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 5)                 505       
                                                                 
Total params: 26,505
Trainable params: 26,505
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200


ValueError: ignored

In [ ]:
metric = tfa.metrics.F1Score(num_classes=n_classes)
metric.update_state(y_test, model.predict(x_test))
result = metric.result()
f1 = result.numpy()
print(f1, f1.mean())

454/454 [==============================] - 3s 4ms/step
[0.99175483 0.5662651  0.70119524 0.77477473 0.55399054] 0.7175961


In [ ]:
def create_model():
  inputs = layers.Input(shape=(24,20), name='input')
  x1 = layers.Bidirectional(layers.LSTM(units=50,return_sequences=False))(inputs)
  x1 = layers.Dropout(0.2)(x1)
  x2 = layers.Bidirectional(layers.LSTM(units=50,return_sequences=False))(inputs)
  x2 = layers.Dropout(0.2)(x2)
  outputs = layers.Dense(n_classes, activation="softmax")(x2)
  return keras.models.Model(inputs=inputs, outputs=outputs)
model = create_model()
model.load_weights("/content/drive/MyDrive/training_1/cp.ckpt")


In [ ]:
features.columns

Index(['datetime', 'machineID', 'volt', 'rotate', 'pressure', 'vibration',
       'errorID_error1', 'errorID_error2', 'errorID_error3', 'errorID_error4',
       'errorID_error5', 'comp_comp1', 'comp_comp2', 'comp_comp3',
       'comp_comp4', 'age', 'model_model1', 'model_model2', 'model_model3',
       'model_model4', 'hours', 'failure'],
      dtype='object')

In [ ]:
features.iloc[0].values

array([Timestamp('2015-01-01 06:00:00'), 1, 176.217853015625,
       418.504078221616, 113.077935462083, 45.0876857639276, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18, 0, 0, 1, 0, 0.25, 0],
      dtype=object)

In [ ]:
features["failure"].value_counts()

0    876087
2       386
1       291
4       255
3       190
Name: failure, dtype: int64